In [1]:
# Import dependencies
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import timeit

from datetime import datetime
from config import weather_api_key

In [2]:
file_path="Resources/Electric_Vehicle_Charging_Stations_in_New_York.csv"
Charger_df=pd.read_csv(file_path)
Charger_df

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,Latitude,Longitude,Date Last Confirmed,ID,Updated At,Owner Type Code,Federal Agency ID,Federal Agency Name,Open Date,EV Connector Types
0,ELEC,Town of Penfield - Library,1985 Baird Rd,NaN,Penfield,NY,14526,NaN,585-340-8720,E,...,43.137476,-77.465214,06/07/2021,49941,2022-02-10 19:42:29,LG,NaN,NaN,04/01/2012,J1772
1,ELEC,Village of Webster,28 West Main Street,NaN,Webster,NY,14580,NaN,866-816-7584,E,...,43.211386,-77.431663,11/10/2022,116756,2022-11-10 02:00:34,NaN,NaN,NaN,08/31/2018,J1772
2,ELEC,HOTSARA CHARGER STATION 1,100 Main St,NaN,Saranac Lake,NY,12983,NaN,888-758-4389,E,...,44.326862,-74.130150,11/10/2022,149787,2022-11-10 00:57:46,NaN,NaN,NaN,12/07/2019,J1772
3,ELEC,Houghton College,9736 Rt 19,NaN,Houghton,NY,14744,NaN,866-816-7584,E,...,42.423869,-78.154428,11/10/2022,157942,2022-11-10 02:00:45,NaN,NaN,NaN,05/15/2020,J1772
4,ELEC,NYC FLEET DPRMBLOT8_1_L3,920 Father Capodanno Blvd,NaN,Midland Beach,NY,10305,NaN,888-758-4389,E,...,40.572636,-74.086140,11/10/2022,182295,2022-11-10 01:05:12,NaN,NaN,NaN,01/27/2021,CHADEMO J1772COMBO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3079,ELEC,W DELAVAN DELAVAN (1),722 W Delavan Ave,NaN,Buffalo,NY,14222,NaN,888-758-4389,E,...,42.922743,-78.873098,11/10/2022,154370,2022-11-10 01:01:48,NaN,NaN,NaN,03/07/2020,J1772
3080,ELEC,HAMPTON INN STATION 1,1120 South Ave,NaN,Staten Island,NY,10314,NaN,888-758-4389,E,...,40.612696,-74.179012,11/10/2022,168146,2022-11-10 01:05:20,NaN,NaN,NaN,11/05/2020,J1772
3081,ELEC,SRYMCASSB BATTENKILL 1.1,15 Van Ness Ave,NaN,Greenwich,NY,12834,NaN,888-758-4389,E,...,43.093964,-73.506775,11/10/2022,183460,2022-11-10 01:06:06,NaN,NaN,NaN,01/27/2021,J1772
3082,ELEC,PAUL MILLER VW VW_408107_A,5865 Rome-Taberg Rd,NaN,Rome,NY,13440,NaN,888-758-4389,E,...,43.233239,-75.497597,11/10/2022,152393,2022-11-10 00:42:37,NaN,NaN,NaN,01/11/2020,J1772


In [3]:
Charger_df.columns


Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count',
       'EV Other Info', 'EV Network', 'EV Network Web', 'Geocode Status',
       'Latitude', 'Longitude', 'Date Last Confirmed', 'ID', 'Updated At',
       'Owner Type Code', 'Federal Agency ID', 'Federal Agency Name',
       'Open Date', 'EV Connector Types'],
      dtype='object')

In [26]:
Charger_df = Charger_df [['ID','Station Name', 'City', 'ZIP']]
Charger_df

,ID,Station Name,City,ZIP
0,49941,Town of Penfield - Library,Penfield,14526
1,116756,Village of Webster,Webster,14580
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983
3,157942,Houghton College,Houghton,14744
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305
...,...,...,...,...
3079,154370,W DELAVAN DELAVAN (1),Buffalo,14222
3080,168146,HAMPTON INN STATION 1,Staten Island,10314
3081,183460,SRYMCASSB BATTENKILL 1.1,Greenwich,12834
3082,152393,PAUL MILLER VW VW_408107_A,Rome,13440


In [45]:
Charger_df['City'].nunique()

570

In [7]:
pip install uszipcode

  Using cached uszipcode-1.0.1-py2.py3-none-any.whl (35 kB)
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
  Using cached sqlalchemy_mate-1.4.28.3-py2.py3-none-any.whl (76 kB)
  Using cached haversine-2.7.0-py2.py3-none-any.whl (6.9 kB)
  Using cached pathlib_mate-1.1.1-py2.py3-none-any.whl (121 kB)
  Using cached prettytable-3.5.0-py3-none-any.whl (26 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from uszipcode import SearchEngine

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
#Creating a dictionary that shows which county each zipcode is in
zipcodes_to_search = list(Charger_df['ZIP'].unique())
search = SearchEngine()
zip_map = {}

# excluding the zipcode not corresponded to any county
for zipcode in zipcodes_to_search:
    if zipcode == 12081:
        pass
    elif zipcode == 12217:
        pass
    else:
        zip_info = search.by_zipcode(zipcode).county
        zip_map[zipcode] = zip_info

zip_map

{14526: 'Monroe County',
 14580: 'Monroe County',
 12983: 'Franklin County',
 14744: 'Allegany County',
 10305: 'Richmond County',
 14227: 'Erie County',
 11220: 'Kings County',
 13090: 'Onondaga County',
 10954: 'Rockland County',
 14853: 'Tompkins County',
 11040: 'Nassau County',
 12205: 'Albany County',
 11976: 'Suffolk County',
 12590: 'Dutchess County',
 11212: 'Kings County',
 12534: 'Columbia County',
 14204: 'Erie County',
 10901: 'Rockland County',
 12180: 'Rensselaer County',
 10065: 'New York County',
 13027: 'Onondaga County',
 14621: 'Monroe County',
 11954: 'Suffolk County',
 11710: 'Nassau County',
 12110: 'Albany County',
 11901: 'Suffolk County',
 12020: 'Saratoga County',
 12477: 'Ulster County',
 13441: 'Oneida County',
 12144: 'Rensselaer County',
 14223: 'Erie County',
 14850: 'Tompkins County',
 14616: 'Monroe County',
 12054: 'Albany County',
 12866: 'Saratoga County',
 14731: 'Cattaraugus County',
 10022: 'New York County',
 14072: 'Erie County',
 12065: 'Sarat

In [27]:
#mapping county information for each charger based on zipcode
Charger_df['County'] = Charger_df['ZIP'].map(zip_map) 
Charger_df.head()

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,Station Name,City,ZIP,County
0,49941,Town of Penfield - Library,Penfield,14526,Monroe County
1,116756,Village of Webster,Webster,14580,Monroe County
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983,Franklin County
3,157942,Houghton College,Houghton,14744,Allegany County
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305,Richmond County


In [28]:
#Creating a dictionary that shows which latitude each zipcode is in
zipcodes_to_search = list(Charger_df['ZIP'].unique())
search = SearchEngine()
lat_map = {}

# excluding the zipcode not corresponded to any county
for zipcode in zipcodes_to_search:
    if zipcode == 12081:
        pass
    elif zipcode == 12217:
        pass
    else:
        lat_info = search.by_zipcode(zipcode).lat
        lat_map[zipcode] = lat_info

lat_map

{14526: 43.15,
 14580: 43.21,
 12983: 44.32,
 14744: 42.42,
 10305: 40.6,
 14227: 42.89,
 11220: 40.64,
 13090: 43.15,
 10954: 41.09,
 14853: 42.45,
 11040: 40.73,
 12205: 42.72,
 11976: 40.92,
 12590: 41.59,
 11212: 40.66,
 12534: 42.25,
 14204: 42.88,
 10901: 41.11,
 12180: 42.73,
 10065: 40.76,
 13027: 43.15,
 14621: 43.19,
 11954: 41.04,
 11710: 40.65,
 12110: 42.74,
 11901: 40.94,
 12020: 43.0,
 12477: 42.07,
 13441: 43.23,
 12144: 42.64,
 14223: 42.97,
 14850: 42.44,
 14616: 43.23,
 12054: 42.62,
 12866: 43.06,
 14731: 42.27,
 10022: 40.76,
 14072: 43.01,
 12065: 42.85,
 11766: 40.93,
 13326: 42.7,
 12561: 41.74,
 11705: 40.74,
 12203: 42.68,
 10025: 40.8,
 11415: 40.71,
 12204: 42.69,
 14210: 42.88,
 10314: 40.6,
 10028: 40.78,
 10523: 41.06,
 10013: 40.72,
 14830: 42.14,
 12202: 42.63,
 11732: 40.84,
 12211: 42.7,
 10069: 40.78,
 14564: 42.98,
 10573: 41.02,
 10033: 40.85,
 13413: 43.07,
 12817: 43.63,
 14623: 43.09,
 12970: 44.43,
 14617: 43.23,
 12569: 41.74,
 14612: 43.27,
 

In [31]:
Charger_df['lat'] = Charger_df['ZIP'].map(lat_map) 
Charger_df

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Station Name,City,ZIP,County,lat
0,49941,Town of Penfield - Library,Penfield,14526,Monroe County,43.15
1,116756,Village of Webster,Webster,14580,Monroe County,43.21
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983,Franklin County,44.32
3,157942,Houghton College,Houghton,14744,Allegany County,42.42
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305,Richmond County,40.60
...,...,...,...,...,...,...
3079,154370,W DELAVAN DELAVAN (1),Buffalo,14222,Erie County,42.92
3080,168146,HAMPTON INN STATION 1,Staten Island,10314,Richmond County,40.60
3081,183460,SRYMCASSB BATTENKILL 1.1,Greenwich,12834,Washington County,43.08
3082,152393,PAUL MILLER VW VW_408107_A,Rome,13440,Oneida County,43.21


In [34]:
#Creating a dictionary that shows which latitude each zipcode is in
zipcodes_to_search = list(Charger_df['ZIP'].unique())
search = SearchEngine()
lng_map = {}

# excluding the zipcode not corresponded to any county
for zipcode in zipcodes_to_search:
    if zipcode == 12081:
        pass
    elif zipcode == 12217:
        pass
    else:
        lng_info = search.by_zipcode(zipcode).lng
        lng_map[zipcode] = lng_info

lng_map

{14526: -77.44,
 14580: -77.42,
 12983: -74.13,
 14744: -78.16,
 10305: -74.07,
 14227: -78.73,
 11220: -74.02,
 13090: -76.21,
 10954: -74.01,
 14853: -76.48,
 11040: -73.68,
 12205: -73.83,
 11976: -72.34,
 12590: -73.91,
 11212: -73.91,
 12534: -73.78,
 14204: -78.86,
 10901: -74.14,
 12180: -73.67,
 10065: -73.96,
 13027: -76.33,
 14621: -77.6,
 11954: -71.94,
 11710: -73.52,
 12110: -73.74,
 11901: -72.67,
 12020: -73.85,
 12477: -73.94,
 13441: -75.41,
 12144: -73.73,
 14223: -78.85,
 14850: -76.5,
 14616: -77.66,
 12054: -73.83,
 12866: -73.77,
 14731: -78.67,
 10022: -73.97,
 14072: -78.96,
 12065: -73.8,
 11766: -73.01,
 13326: -74.92,
 12561: -74.08,
 11705: -73.05,
 12203: -73.85,
 10025: -73.97,
 11415: -73.83,
 12204: -73.73,
 14210: -78.85,
 10314: -74.17,
 10028: -73.95,
 10523: -73.82,
 10013: -74.0,
 14830: -77.05,
 12202: -73.76,
 11732: -73.54,
 12211: -73.76,
 10069: -73.99,
 14564: -77.41,
 10573: -73.68,
 10033: -73.93,
 13413: -75.28,
 12817: -73.8,
 14623: -77.6

In [35]:
Charger_df['lng'] = Charger_df['ZIP'].map(lng_map) 
Charger_df

,ID,Station Name,City,ZIP,County,lat,lng
0,49941,Town of Penfield - Library,Penfield,14526,Monroe County,43.15,-77.44
1,116756,Village of Webster,Webster,14580,Monroe County,43.21,-77.42
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983,Franklin County,44.32,-74.13
3,157942,Houghton College,Houghton,14744,Allegany County,42.42,-78.16
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305,Richmond County,40.60,-74.07
...,...,...,...,...,...,...,...
3079,154370,W DELAVAN DELAVAN (1),Buffalo,14222,Erie County,42.92,-78.88
3080,168146,HAMPTON INN STATION 1,Staten Island,10314,Richmond County,40.60,-74.17
3081,183460,SRYMCASSB BATTENKILL 1.1,Greenwich,12834,Washington County,43.08,-73.49
3082,152393,PAUL MILLER VW VW_408107_A,Rome,13440,Oneida County,43.21,-75.47


In [36]:
lats = list(Charger_df['lat'])
lngs = list(Charger_df['lng'])

# pack the random coordinates into a zip
coordinates_list = zip(lats,lngs)

coordinates_list

In [37]:
coordinates= list(coordinates_list)
print(coordinates)

[(43.15, -77.44), (43.21, -77.42), (44.32, -74.13), (42.42, -78.16), (40.6, -74.07), (42.89, -78.73), (40.64, -74.02), (43.15, -76.21), (41.09, -74.01), (42.45, -76.48), (40.73, -73.68), (42.72, -73.83), (40.92, -72.34), (41.59, -73.91), (40.66, -73.91), (42.25, -73.78), (42.88, -78.86), (41.11, -74.14), (42.73, -73.67), (40.76, -73.96), (43.15, -76.33), (43.19, -77.6), (41.04, -71.94), (42.25, -73.78), (40.65, -73.52), (42.74, -73.74), (40.94, -72.67), (43.0, -73.85), (42.07, -73.94), (43.23, -75.41), (42.64, -73.73), (42.97, -78.85), (44.32, -74.13), (42.44, -76.5), (43.23, -77.66), (42.62, -73.83), (42.42, -78.16), (43.06, -73.77), (42.27, -78.67), (43.06, -73.77), (40.76, -73.97), (43.01, -78.96), (42.85, -73.8), (40.93, -73.01), (42.7, -74.92), (41.74, -74.08), (40.74, -73.05), (42.68, -73.85), (40.8, -73.97), (40.71, -73.83), (42.69, -73.73), (42.88, -78.85), (40.6, -74.17), (40.78, -73.95), (41.06, -73.82), (40.72, -74.0), (42.14, -77.05), (42.63, -73.76), (40.84, -73.54), (42.7

In [38]:
# Get the nearest city using the citipy module
# Create a list to hold the nearest cities
nearest_cities=[]

# detrmining the nearest city for each pair of lats and lngs
for coordinate in coordinates:
    city=citipy.nearest_city(coordinate[0], coordinate[1]).city_name
#including unique city in the list
    if city not in nearest_cities:
        nearest_cities.append(city)
#printing the city count to see how many cities we have in the list

len(nearest_cities)

169

In [39]:
#Perform an API call with the OpenWeatherMap.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [40]:
import time
from datetime import datetime
# Create an empty list to hold the weather data.
city_data = []
# indicating the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(nearest_cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(55)
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_info = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_info["coord"]["lat"]
        city_lng = city_info["coord"]["lon"]
        city_country = city_info["sys"]["country"]
        city_max_temp = city_info["main"]["temp_max"]
        city_min_temp = city_info["main"]["temp_min"]
        city_humidity = city_info["main"]["humidity"]
        city_clouds = city_info["clouds"]["all"]
        city_wind = city_info["wind"]["speed"]
        description = city_info['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_info["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Min Temp": city_min_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | brighton
Processing Record 2 of Set 1 | huntingdon
Processing Record 3 of Set 1 | olean
Processing Record 4 of Set 1 | bayonne
Processing Record 5 of Set 1 | cheektowaga
Processing Record 6 of Set 1 | new york
Processing Record 7 of Set 1 | fairmount
Processing Record 8 of Set 1 | nanuet
Processing Record 9 of Set 1 | ithaca
Processing Record 10 of Set 1 | new hyde park
Processing Record 11 of Set 1 | albany
Processing Record 12 of Set 1 | old saybrook
Processing Record 13 of Set 1 | beacon
Processing Record 14 of Set 1 | kingston
Processing Record 15 of Set 1 | buffalo
Processing Record 16 of Set 1 | suffern
Processing Record 17 of Set 1 | troy
Processing Record 18 of Set 1 | guttenberg
Processing Record 19 of Set 1 | irondequoit
Processing Record 20 of Set 1 | groton
Processing Record 21 of Set 1 | bellmore
Processing Record 22 of Set 1 | watervliet
Processing Record 23 of Set 1 | mastic
Proces

In [41]:
len (city_data)

169

In [43]:
# adding  the weather data to a new DataFrame
city_data_df=pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Brighton,GB,50.8284,-0.1395,52.75,44.35,94,100,18.99,overcast clouds
1,Huntingdon,GB,52.3305,-0.1865,46.02,41.79,93,100,11.68,overcast clouds
2,Olean,US,42.0776,-78.4297,23.65,21.96,68,100,8.05,overcast clouds
3,Bayonne,FR,43.3333,-1.4167,51.13,45.55,87,75,2.30,broken clouds
4,Cheektowaga,US,42.9034,-78.7548,24.78,19.72,84,100,7.00,overcast clouds


In [44]:
# Exporting the DataFrame as WeatherPy_Database.csv into the Weather_Database folder
# defining the path
output_data_path = "Resources/WeatherPy_Database.csv"

# exporting data into csv file
city_data_df.to_csv(output_data_path, index_label="city_id")